[dataset: FSD50k (Free Sound Dataset 50k)](https://zenodo.org/record/4060432#.X5ySHHhKgWo)


Thesis: Classify raw audio files into categories associated with household noises to identify emergency related sounds(humans (voice), animals, instruments, water, door, car, explosion (boom, thunder).

### 3 Main CNN Models:

1. CNN with Wave Spectrograms
2. CNN with MFCC Spectrograms
3. CNN with Mean Values of MFCC Features

for graphing keras model results

also start random forest some point soon (or xgboost? or both?)

In [ ]:
# import os

# list = os.listdir('/home/ec2-user/SageMaker/data/train_mfcc') # dir is your directory path
# number_files = len(list)
# print(number_files)

In [ ]:
# run with --yes
# !conda install -c conda-forge librosa 

In [2]:
import pandas as pd
import numpy as np
import time
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
from PIL import Image
import skimage.io
import os
import pickle
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras import optimizers

%matplotlib inline

In [ ]:
def pred_checker(track_num):
    pred_index = np.argmax(preds[track_num])

    # checking what the audio file is
    print("Description of Audio file:", df_2000.iloc[track_num]['description'])

    print('\nPredicted Class:', label_dict[pred_index],'\nTrue Class:', label_dict[df_2000.iloc[track_num]['labels']])

    # Lets play the audio 

    file_path = audio_dir + list_of_wav_names[track_num]
    data, sr = librosa.load(file_path)

    sound = ipd.Audio(file_path)
    return sound

# pred_checker(44)
# #  for fuller track info: df_2000.iloc[track_num]

### Loading Meta Data

In [3]:
dev_info = pd.read_json('data/labelled_dev_info.json')
eval_info = pd.read_json('data/labelled_eval_info.json')

In [4]:
dev_info.head(2)

,title,description,tags,license,uploader,track_num,wav_name,png_name,labels_15,labels_2,labels_4,labels
63,RalfHutterWorking.wav,"Ralf Hutter from Kraftwerk saying ""Working on ...","[male, voice]",http://creativecommons.org/licenses/by/3.0/,fectoper,63,63.wav,63.png,0,0,0,0
136,keyboard-rhymtic.wav,Noise of an average logitech keyboard. Pretty ...,"[keyboard, rhythmic, tap, type]",http://creativecommons.org/licenses/by/3.0/,Anton,136,136.wav,136.png,11,8,3,1


In [5]:
eval_info.head(2)

,title,description,tags,license,uploader,track_num,wav_name,png_name,labels_15,labels_2,labels_4,labels
391277,Spring Birds Forest 04 Amp.wav,An other birds singings recorded on the mornin...,"[birdsong, bird, forest, environment, morning,...",http://creativecommons.org/publicdomain/zero/1.0/,ANARKYA,391277,391277.wav,391277.png,4,2,1,2
392115,Snap of fingers,a snap of one's fingers,"[fingers, finger, 5maudio17, uam, fingersnap]",http://creativecommons.org/publicdomain/zero/1.0/,edton,392115,392115.wav,392115.png,6,8,3,1


# Begin MODELING

In [6]:
data_dir = '/home/ec2-user/SageMaker/data/'
# not uploading audio to sagemaker
# audio_dir = '/Users/Mateo/Springboard/FSD50k/data/FSD50K.dev_audio/'
train_wav_names = dev_info.wav_name.to_list()
train_png_names = dev_info.png_name.to_list()

test_wav_names = eval_info.wav_name.to_list()
test_png_names = eval_info.png_name.to_list()

In [6]:
# Old dict

# label_dict ={
#     0: 'human',
#     1: 'comedy',
#     2: 'door',
#     3: 'baby',
#     4: 'animal',
#     5: 'music',
#     6: 'percussion',
#     7: 'machines',
#     8: 'ambient',
#     9: 'fire',
#     10: 'movement',
#     11: 'household',
#     12: 'foley',
#     13: 'nature',
#     14: 'other'    
# }

## Sanity Check For Mini Dataframe (2000)

In [8]:
df_2000 = dev_info.copy() # no slice!

In [9]:
df_2000.shape

(40966, 12)

# 1. CNN from Spectogram (move to first position)


### Notes on Architecture:(https://github.com/GorillaBus/urban-audio-classifier/blob/master/3-cnn-model-mfcc.ipynb)
    
   Convolutional layers and filter amounts: each convolutional layer will learn different features from the training data. The first layers will always learn more low level features (like lines and dots on an image, for example), while the following layers will learn more high-level patterns (basic shapes). The more complex the data is in means of patterns, the more convolutional layers that will be required learn them.
On the other hand, there will always be less low-level patterns and more high-level patters (formed by combinations of those lower level patterns), that's why we keep increasing the amount of filters as we add convolutional layers to the network.

Kernel sizes: I'm using one 5x5 and 3x3 kernel sizes as the smallest size that worked out on my experiments. I also tested 7x7 -as saw on some image classifiers- but they didn't work well with this data. Also notice I use odd numbers for kernel sizes, this is a very important rule and here is an excellent explanation Stackoverflow.

Batch normalization: In short, it normalizes the output of a previous activation layer by subtracting the batch mean and dividing by the batch standard deviation. This process optimizes training time. There is a friendly introduction to batch normalization .

MaxPooling: a common technique to downsample our data reducing dimensionality, if not familiarized have a look at this article from computersciencewiki.org.

Spatial dropout: A regularization method that works like the standard dropout used in neural networks, that applies better for convolutional layers: it works by dropping out entire feature maps by a given rate, preventing activations from becoming strongly correlated. It's effect is usually more sensible than the standard dropout and as you may have advertised I'm increasing the rate as I add new layers, this is a common practice as higher level layers are also containing more filters. Another important advice is to use it after max pooling.

GlobalAveragePooling: until some years ago it was mostly common to see CNN architectures that would flatten out the output from the last convolutional layer to connect it to a series of dense layers that would "interprete" and make category predictions. Today is more common to see this dense layers replaced with a Global Average Pooling layer that calculates the average output of each feature map in the previous layer, strongly reducing dimensionality. This demonstrated to work better in many scenarios, also being much more computationally cheap. Read more in this article.

Dense (softmax output): the final layer containing the softmax output that will provide the classification probabilities for the input data.


### Notes on Keras/ Documentation

Conv2D documentation: https://keras.io/api/layers/convolution_layers/convolution2d/
MaxPooling2D: https://keras.io/api/layers/pooling_layers/max_pooling2d/
Dropout: https://keras.io/api/layers/regularization_layers/dropout/
Flatten: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
Dense: https://keras.io/api/layers/core_layers/dense/


Conv2D
the first number (32) represents 'filters', or how many outputs for this layer (that's why it grows for each Conv2D)
- these filters are the same as nodes (maybe not)
kernel_size = size of sliding window
default: strides = (1,1)
default: padding = 'valid' (means 'none', switch to 'same' for equal padding for all kernels/windows)

MaxPooling2D:
pool_size: dimensions of pixels to analyze, aggegrating by the max value
strides = default, switch uses dimensions of pool_size for sliding window (thus does not aggegrate overlapping pixels)
padding = default, which is 'valid'(none), 'same' creates equal padding (better no?)

Dropout:
Fraction of the input units to drop.
The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time
I believe that means that one designates the percentage of input dropped, that are randomly (not evenly) spaced?

Flatten
takes a multi-dimensioned matrix, and converts it into a 1-d array.
i.e. An output from a Conv2D layer with a shape of (1,10,64) passed through Flatten would have a shape of (640,)

Dense:
Just your regular densely-connected NN layer. 
-(a fully connected layer? as opposed to a Convutional layer, which is like a sliding window?)
units: Positive integer, dimensionality of the output space.
activation: Activation function to use. Commonly 'relu' when a binary output is desired
-softmax is generally used for the final output layer on multiclassification problems).


Optimizer/ Compile:
alternate Optimizers in compiling (as opposed to SGD Stochastic Gradient Descent)
(note: also use sparse_categorical_crossentropy with numeric input for label in .flow_from_dataframe)

optimizer = Adam(lr=1e-4, beta_1=0.99, beta_2=0.999)
optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
optimizer = keras.optimizers.adam()
optimizer = optimizers.SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)



Note: Consider adding a Keras BatchNormilization layer (between linear and nonlinear layers, after flatten?)


Feature dimensions (MFCC x Time)


### loading ONE image from Spectrogram directory 

In [9]:
eval_info.labels.nunique()

8

In [ ]:
# # loading Spectrogram (sanity check)

# spec_images_dir = data_dir + 'train_spectrograms/'
# file_path = spec_images_dir + train_png_names[0]

# # loading processed MFCC Spectrogram Images
# spec = skimage.io.imread(file_path)
# print(spec.shape)
# librosa.display.specshow(spec)

# plt.ylabel('MFCC Coefficient Bands')
# plt.xlabel('Time in 23 millisecond increments')
# plt.title('Spectrogram of Audio Wave')
# plt.yticks(range(0,128, 24))
# plt.xticks(range(0,217, 48))
# plt.colorbar()
# plt.title('Spectrogram of Audio Wave')

# plt.show()

In [ ]:
# load data into generator with keras function .flow_from_directory

datagen=ImageDataGenerator(rescale=1./255.,
                          validation_split=0.1)  # , default: validation_split= None
# 25% test set before, thats why only 75 before, seperate  eval so validiation =0


train_spec_generator=datagen.flow_from_dataframe(
    dataframe= df_2000,
    directory= 'data/train_spectrograms/',
    x_col='png_name',
    y_col= 'labels', # previously set as subset="training", use when having validation_split
    color_mode="grayscale",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode=  "raw", #  'raw' for numeric input
    target_size=(128,216),
    subset='training') # target_size=(64,64)) tuple of height/width

valid_spec_generator=datagen.flow_from_dataframe(
    dataframe= df_2000,
    directory= 'data/train_spectrograms',
    x_col='png_name',
    y_col= 'labels', # previously set as subset="training", use when having validation_split
    color_mode="grayscale",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode=  "raw", #  'raw' for numeric input
    target_size=(128,216),
    subset='validation') # target_size=(64,64)) tuple of height/width

# test generator
datagen=ImageDataGenerator(rescale=1./255.)

test_spec_generator=datagen.flow_from_dataframe(
    dataframe= eval_info,
    directory= 'data/test_spectrograms',
    x_col='png_name',
    y_col= 'labels',
    color_mode="grayscale",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode=  "raw", # 'raw' for numeric input
    target_size=(128,216)) # target_size=(64,64)) tuple of height/width

In [ ]:
num_classes= 8 # dev_info.labels.nunique()

#Define Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128,216,1))) # Height, Width, Num_channels
# seems like this should be just 1 channel, but asked for three, check this out later
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# optimizer=tf.keras.optimizers.adam() didnt work...
optimizer = optimizers.SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)


#Compile   # had to switch to sparse_categorical_crossentropy from categorical_crossentropy
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

print(model.summary())

In [ ]:
#Train and Test The Model
model.fit(x=train_spec_generator,validation_data=valid_spec_generator, epochs=3, verbose=1)    # switched to 5 from 10

In [ ]:
# Evaluating Testing Data

score = model.evaluate(test_spec_generator)
accuracy = 100*score[1]
print("Testing accuracy: %.2f%%" % accuracy) 

# 2. CNN with MFCC Spectrogram

In [ ]:
# # loading one MFCC Spectrogram

# spec_images_dir = data_dir + 'train_mfcc/'
# file_path = spec_images_dir + train_png_names[0]


# # loading processed MFCC Spectrogram Images
# spec = skimage.io.imread(file_path)
# print(spec.shape)
# librosa.display.specshow(spec)

# plt.ylabel('MFCC Coefficient Bands')
# plt.xlabel('Time in 23 millisecond increments')
# plt.title('MFCC Spectrogram')
# plt.yticks(range(1,33))
# plt.xticks(range(0,217, 48))
# plt.colorbar()

# plt.show()

In [10]:
# loading MFCC Spectrograms into generator 

datagen=ImageDataGenerator(rescale=1./255.,
                          validation_split=0.1)  


mfcc_train_generator=datagen.flow_from_dataframe(
    dataframe= df_2000,
    directory= 'data/train_mfcc',
    x_col='png_name',
    y_col= 'labels',
    color_mode="grayscale",
    batch_size=32,
    seed=42,
    shuffle=True,
    subset='training',
    class_mode="raw", 
    target_size=(32,216)) # target_size=(64,64)) tuple of height/width

mfcc_valid_generator=datagen.flow_from_dataframe(
    dataframe= df_2000,
    directory= 'data/train_mfcc',
    x_col='png_name',
    y_col= 'labels',
    color_mode="grayscale",
    batch_size=32,
    seed=42,
    shuffle=True,
    subset='validation',
    class_mode="raw",
    target_size=(32,216)) #tuple of height/width

# test generator
datagen=ImageDataGenerator(rescale=1./255.)

mfcc_test_generator=datagen.flow_from_dataframe(
    dataframe= eval_info,
    directory= 'data/test_mfcc',
    x_col='png_name',
    y_col= 'labels',
    color_mode="grayscale",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw", 
    target_size=(32,216)) # target_size=(64,64)) tuple of height/width

Found 36870 validated image filenames.
Found 4096 validated image filenames.
Found 10231 validated image filenames.


In [11]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(32, 216, 1))) # Height, Width, num_channels
model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))  # num_classes = 15

optimizer = optimizers.SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 215, 32)       160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 214, 48)       6192      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 213, 120)      23160     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 106, 120)      0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 106, 120)      0         
_________________________________________________________________
flatten (Flatten)            (None, 178080)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               2

In [ ]:
model.fit(x=mfcc_train_generator, validation_data=mfcc_valid_generator, epochs= 20, verbose=1)    # switched to 5 from 10

Epoch 1/20
1153/1153 [==============================] - 1592s 1s/step - loss: 1.3500 - accuracy: 0.5246 - val_loss: 1.3975 - val_accuracy: 0.5107
Epoch 4/20
 448/1153 [==========>...................] - ETA: 15:27 - loss: 1.3007 - accuracy: 0.5410

In [ ]:
# Evaluating the model on the training and testing set

score = model.evaluate(mfcc_test_generator, verbose=1)
accuracy = 100*score[1]
print("Testing accuracy: %.2f%%" % accuracy) 

# score = model.evaluate(x_test, y_test, verbose=0)
# print("Testing Accuracy: ", score[1])

# 3. CNN with Mean MFCC Values

In [ ]:
# loading Training and Testing Data: Mean MFCC features
    
train_mfcc_mean = np.load('data/train_mean_mfcc.npz')['arr_0']
test_mfcc_mean = np.load('data/test_mean_mfcc.npz')['arr_0']

print(train_mfcc_mean.shape)
print(test_mfcc_mean.shape)
# sample mean_mfcc_array:
train_mfcc_mean[0]

In [ ]:
y_train = dev_info['labels'] # to_categorical(dev_info['labels']) # no slice
y_test = eval_info['labels'] # to_categorical(eval_info['labels']) # no slice

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(15, activation='softmax'))

In [ ]:
# compile the keras model
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_mfcc_mean, y_train, epochs=10, batch_size=50, verbose=1)

In [ ]:
# Evaluating the model on the training and testing set

score = model.evaluate(train_mfcc_mean, y_train, verbose=1)
accuracy = 100*score[1]
print("Training accuracy: %.2f%%" % accuracy) 

score = model.evaluate(test_mfcc_mean, y_test, verbose=1)
accuracy = 100*score[1]
print("Test accuracy: %.2f%%" % accuracy) 

## MFCC CNN without generator



In [ ]:
train_mfcc = pd.read_json('data/train_mfcc_df.json')


In [ ]:
test_mfcc = pd.read_json('data/test_mfcc_df.json')

In [ ]:
train_mfcc.head()

In [ ]:
test_mfcc

In [ ]:
X_train = train_mfcc['mfcc']
X_test = test_mfcc['mfcc']

In [ ]:
y_train = train_mfcc['labels'] # to_categorical(dev_info['labels']) # no slice
y_test = test_mfcc['labels'] # to_categorical(eval_info['labels']) # no slice

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(32, 216, 1))) # Height, Width, num_channels
model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))  # num_classes = 15

optimizer = optimizers.SGD(lr=0.002, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=50, verbose=1)

### Note: read some articles on extracting feature importance from this basic neural net
### Then maybe a Random Forest Model to then get performance + feat.importance/interpretable results

- evaluate model performance by shuffling bands and evaluating affect on model performance (PERMUTATION FEAT. IMPORT)
(https://christophm.github.io/interpretable-ml-book/feature-importance.html)

- Random Forest Feature Importance (Impurity based)(Warning: impurity-based feature importances can be misleading for high cardinality features (many unique values). See sklearn.inspection.permutation_importance as an alternative.)
(https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_)

- XGBoost Classifier Feature Importance (https://stackoverflow.com/questions/37627923/how-to-get-feature-importance-in-xgboost)
(https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/)
-https://mljar.com/blog/feature-importance-xgboost/

-(also consider Drop Column Feature Importance)
https://explained.ai/rf-importance/index.html

## End | Notes Below

In [ ]:
# This trims silence???? use maybe on longer audio tracks?

# trim silent edges

whale_song, _ = librosa.effects.trim(y)

In [ ]:
  # labeling pickle files instructions:
    
    # store extracted training data
    training_examples.to_pickle('Extracted_Features\\' + training_name + '_features.pkl')
    training_labels.to_pickle('Extracted_Features\\' + training_name + '_labels.pkl')

    # store extracted validation data
    validation_examples.to_pickle('Extracted_Features\\' + validation_name + '_features.pkl')
    validation_labels.to_pickle('Extracted_Features\\' + validation_name + '_labels.pkl')
    
    # import glob
# import imageio

# for image_path in glob.glob("/Users/Mateo/Springboard/FSD50k_stuff/data/spec_100/*.png"):
#     im = imageio.imread(image_path)
#     plt.imshow(im)

In [ ]:
# 1D CNN Not Applicable for this project (would apply if analyzing straight audio wav files (better for timeseries))

# from keras.layers import Conv1D
# from keras.layers import MaxPooling1D

# # https://keras.io/api/layers/convolution_layers/convolution1d/
# # https://keras.io/api/layers/pooling_layers/max_pooling1d/

# verbose, epochs, batch_size = 0, 10, 32

# n_timesteps, n_features, n_outputs = 1, 13, 15 # no time, 13 mean features, 15 class outputs of model


# model = Sequential()
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(13)))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(100, activation='relu'))
# model.add(Dense(n_outputs, activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit(mfcc_mean_features, y_hot, epochs=epochs, batch_size=batch_size, verbose=verbose)
# # evaluate model

In [ ]:
# helpful articles

https://towardsdatascience.com/getting-to-know-the-mel-spectrogram-31bca3e2d9d0
https://towardsdatascience.com/audio-classification-using-fastai-and-on-the-fly-frequency-transforms-4dbe1b540f89
https://pytorch.org/audio/stable/index.html
(good explanations of Mel and CNN) https://scottmduda.medium.com/urban-environmental-audio-classification-using-mel-spectrograms-706ee6f8dcc1
    
(indepth explanation of MFS MFCC)    https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html
 free course on audio processing   https://www.coursera.org/learn/audio-signal-processing
    
Urban sound dataset (Consideration, 4seconds in length) https://urbansounddataset.weebly.com/urbansound8k.html
https://medium.com/gradientcrescent/urban-sound-classification-using-convolutional-neural-networks-with-keras-theory-and-486e92785df4
    
    
# audio walk through with keras
http://aqibsaeed.github.io/2016-09-03-urban-sound-classification-part-1/